In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [4]:
c=pd.read_csv('modeling.csv')

In [7]:
c['description']=c['description'].replace('unknowntoken','unknown token')
c['details']=c['details'].replace('unknowntoken','unknown token')
c['brand_category']=c['brand_category'].replace('unknowntoken','unknown token')

,Unnamed: 0,index,product_id,description,details,name,brand_category,is_work,is_nightout,is_daytonight,is_vacation,is_workout,is_weekend,is_coldweather


In [2]:
import gensim
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

In [9]:
def predict_occasion(brand,description,brand_category,wv):
    import warnings
    warnings.filterwarnings("ignore")
    modeling=pd.read_csv('modeling.csv')
    modeling['description']=modeling['description'].replace('unknowntoken','unknown token')
    modeling['details']=modeling['details'].replace('unknowntoken','unknown token')
    modeling['brand_category']=modeling['brand_category'].replace('unknowntoken','unknown token')
    #Clean the query 
    
    def replace(word):
        word=word.replace('/',' ')
        new=re.sub('[^a-zA-Z]', ' ',word)
        p=[]
        for k in new.split():
            separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
            new = separated.lower()
            p.append(new)
        ls=WordNetLemmatizer()
        new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        return new
    
    brand=replace(brand)
    description=replace(description)
    brand_category=replace(brand_category)
    
    
    def word_averaging(wv, words):
        all_words, mean = set(), []

        for word in words:
            if isinstance(word, np.ndarray):
                mean.append(word)
            elif word in wv.vocab:
                mean.append(wv.syn0norm[wv.vocab[word].index])
                all_words.add(wv.vocab[word].index)

        if not mean:
            logging.warning("cannot compute similarity with no input %s", words)
            # FIXME: remove these examples in pre-processing
            return np.zeros(wv.vector_size,)

        mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
        return mean

    def  word_averaging_list(wv, text_list):
        return np.vstack([word_averaging(wv, post) for post in text_list ])

    def w2v_tokenize_text(text):
        tokens = []
        for sent in nltk.sent_tokenize(text, language='english'):
            for word in nltk.word_tokenize(sent, language='english'):
                if len(word) < 2:
                    continue
                tokens.append(word)
        return tokens
    
    c=pd.DataFrame()
    
    
    brand=w2v_tokenize_text(brand)
    description=w2v_tokenize_text(description)
    brand_category=w2v_tokenize_text(brand_category)
    
    brand=word_averaging(wv,brand).reshape(1,-1)
    description=word_averaging(wv,description).reshape(1,-1)
    brand_category=word_averaging(wv,brand_category).reshape(1,-1)
    X4=np.concatenate((brand,description,brand_category),axis=1)
    
    
    names=['description','details','brand_category']
    for i in names:
        X_new = modeling.apply(lambda r: w2v_tokenize_text(r[i]),axis=1).values
        X_word_average = word_averaging_list(wv,X_new)
        X_array=pd.DataFrame(X_word_average)
        c=pd.concat([c,X_array],axis=1)
    tags=['is_work', 'is_nightout', 'is_daytonight', 'is_vacation', 'is_workout',
       'is_weekend', 'is_coldweather']
    for i in tags:
        y=modeling[i].values
        SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='auto')
        SVM.fit(c,y)
        # predict the labels on validation dataset
        result= SVM.predict(X4)
        print(result,i)

In [10]:
desc="blush linen button fastenings along front 100% linen, lining: 100%, cotton dry clean designer color: shell imported"

In [11]:
brand='Zimmermann'

In [12]:
brand_cat='clothing/jumpsuit/full length'

In [13]:
predict_occasion(desc,brand,brand_cat,wv)

[0.] is_work
[0.] is_nightout
[1.] is_daytonight
[0.] is_vacation
[0.] is_workout
[1.] is_weekend
[0.] is_coldweather


# Input

In [ ]:
brand=input("Please enter the brand")
description=input("Please enter description")
brand_cat=input("Please enter brand category")
c=predict_cat(brand,description,brand_cat,wv)
print("The prediction is ",c)